In [17]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report


In [18]:
import tensorflow as tf

# Check for GPU availability
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    print("GPU is available.")
else:
    print("GPU is NOT available.")


GPU is available.


In [19]:
# Define paths for training and testing datasets
train_path = './data/training_set/'
test_path = './data/test_set/'

# Image dimensions and batch size
img_width, img_height = 128, 128
batch_size = 32

# Create image data generators for training and testing
#data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,      # Random rotation within the range [-20, 20] degrees
    width_shift_range=0.1,  # Random horizontal shift by up to 10% of the width
    height_shift_range=0.1, # Random vertical shift by up to 10% of the height
    shear_range=0.2,        # Shear intensity (shear angle in counter-clockwise direction in radians)
    zoom_range=0.2,         # Random zoom up to 20%
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest'     # Strategy used for filling in newly created pixels
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Keep data in the same order as the directory
)


Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [20]:
# Define the CNN model
inputs = Input(shape=(img_width, img_height,3))
x = Conv2D(32, (3, 3), activation='relu', padding="same")(inputs)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(64, (3, 3), activation='relu', padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(128, (3, 3), activation='relu', padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(256, (3, 3), activation='relu', padding="same")(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), strides=(2,2))(x)
x = Conv2D(512, (3, 3), activation='relu', padding="same")(x)
x = Conv2D(512, (3, 3), activation='relu', padding="same")(x)
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
# x = Dense(32, activation='relu')(x)
outputs = Dense(2, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)

# Compile the model
model.compile(optimizer=custom_optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_46 (Conv2D)              │ (None, 128, 128, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 128, 128, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 64, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 64, 64, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_48 (Conv2D)              │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_49 (Conv2D)              │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_50 (Conv2D)              │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_51 (Conv2D)              │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_52 (Conv2D)              │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_53 (Conv2D)              │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_54 (Conv2D)              │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 16, 16, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_55 (Conv2D)              │ (None, 8, 8, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_56 (Conv2D)              │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 5,837,058 (22.27 MB)

 Trainable params: 5,836,098 (22.26 MB)

 Non-trainable params: 960 (3.75 KB)

In [21]:
# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
)

Epoch 1/10
  2/251 ━━━━━━━━━━━━━━━━━━━━ 16s 66ms/step - accuracy: 0.6094 - loss: 0.6859 

W0000 00:00:1713909697.331129     102 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


250/251 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - accuracy: 0.5928 - loss: 0.6648

W0000 00:00:1713909758.448419      99 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


251/251 ━━━━━━━━━━━━━━━━━━━━ 79s 278ms/step - accuracy: 0.5931 - loss: 0.6646 - val_accuracy: 0.4998 - val_loss: 0.7596
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 61s 238ms/step - accuracy: 0.6782 - loss: 0.6009 - val_accuracy: 0.5077 - val_loss: 0.7808
Epoch 3/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 61s 236ms/step - accuracy: 0.7117 - loss: 0.5576 - val_accuracy: 0.7133 - val_loss: 0.5547
Epoch 4/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 61s 236ms/step - accuracy: 0.7381 - loss: 0.5248 - val_accuracy: 0.6683 - val_loss: 0.6156
Epoch 5/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 60s 233ms/step - accuracy: 0.7544 - loss: 0.5097 - val_accuracy: 0.7781 - val_loss: 0.4790
Epoch 6/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 60s 234ms/step - accuracy: 0.7704 - loss: 0.4857 - val_accuracy: 0.7079 - val_loss: 0.5412
Epoch 7/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 63s 245ms/step - accuracy: 0.7764 - loss: 0.4713 - val_accuracy: 0.7815 - val_loss: 0.4609
Epoch 8/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 60s 232ms/step - accuracy: 0.7883 - loss: 0.4596 - val